In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
import os
import requests
import pandas as pd
from io import StringIO
from bs4 import BeautifulSoup
import zipfile
import os
import datetime
import glob
import boto3
from botocore.config import Config
cwd = os.getcwd()
print(cwd)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


/home/ec2-user/SageMaker


In [2]:
# it is necessary to load metadata about stations to start working
os.chdir('/home/ec2-user/SageMaker/')
metadata = pd.read_csv('Metadata-DWD.csv', sep=',', header=0)

In [3]:
# define functions to access data from folder 
def collapse_data_from_folder():
    dir_df = pd.DataFrame()
   
    for file in os.listdir():
        if file.startswith('produkt') and file.endswith('txt'):
            incriment = pd.read_csv(file, sep=';', encoding='ISO-8859-1')
            dir_df = pd.concat([dir_df, incriment])
        else: pass

    return dir_df

def proc_measurements(df):
    df['MESS_DATUM'] = pd.to_datetime(df['MESS_DATUM'], format='%Y%m%d%H')
    #df.set_index('MESS_DATUM', inplace=True)
    if 'eor' in df.columns:
        df.drop(columns = ['eor'], inplace = True)
    return df
#THIS function returns fully prepared dataframe by tag
def prep_from_folder(tag):
    os.chdir(f"/home/ec2-user/SageMaker/basis/{tag}")
    df = collapse_data_from_folder()
    df = proc_measurements(df)
    
    # VERY IMPORTANT: SAMPLING OF WEATHER STATIONS OCCURS HERE 
    df = df[df['STATIONS_ID'].isin(metadata['Stations_id'])]
    # NAs are coded as -999 
    df = df.replace('-999', np.nan).replace(-999, np.nan)
    # filter out only measurements from 2024
    df = df[df['MESS_DATUM'] > pd.to_datetime('2024010100', format='%Y%m%d%H')]    
    return df

#THIS function adapts to other folders like 16Feb
def prep_from_anyfolder(folder, tag):
    os.chdir(f"/home/ec2-user/SageMaker/{folder}/{tag}")
    df = collapse_data_from_folder()
    df = proc_measurements(df)
    
    # VERY IMPORTANT: SAMPLING OF WEATHER STATIONS OCCURS HERE 
    df = df[df['STATIONS_ID'].isin(metadata['Stations_id'])]
    # NAs are coded as -999 
    df = df.replace('-999', np.nan).replace(-999, np.nan)
    # filter out only measurements from 2024
    df = df[df['MESS_DATUM'] > pd.to_datetime('2024010100', format='%Y%m%d%H')]    
    return df

In [4]:
air_temp = prep_from_anyfolder("basis_20Feb","air_temperature")

In [11]:
!pip install mysql-connector-python

In [18]:
#establishing mysql connector 
# this requires instaklling the package
# !pip install mysql-connector-python

import mysql.connector

conn = mysql.connector.connect(
    #database="climadb2",
    user="",
    password="", 
    host=".rds.amazonaws.com",
    port="3306"
)

In [19]:
# print all the names of datbaseson the host
cursor = conn.cursor()
cursor.execute("SHOW DATABASES")
for db in cursor:
    print(db)
cursor.fetchall()

('clima_integra',)
('information_schema',)
('mysql',)
('performance_schema',)
('sys',)


[]

In [20]:
cursor.execute("USE clima_integra")  
cursor.execute("SHOW TABLES")
tables = cursor.fetchall()

for table in tables:
    cursor.execute(f"SELECT COUNT(*) FROM {table[0]}")
    count = cursor.fetchone()[0]
    print(f"Table: {table[0]}, Length: {count}")

Table: DWD, Length: 5995


In [8]:
cursor.fetchall()

[]

In [37]:
# data preparation 
# prepare the big pandas df to ingest the data into timestream
# List of data types
data_types = ['wind', 'sun', 'cloudiness', 'extreme_wind', 'air_temperature', 'moisture', 'precipitation', 'pressure', 'soil_temperature']
# Create a dictionary that maps DWD Name to Measure name
measure_name_dict = {
    'TT_TU': 'air_temperature',
    'RF_TU': 'air_temperature',
    'R1': 'precipitation',
    'WRTR': 'precipitation',
    'V_N': 'cloudiness',
    'SD_SO': 'sun',
    'F': 'wind',
    'D': 'wind',
    'FX_911': 'extreme_wind',
    'ABSF_STD': 'moisture',
    'TF_STD': 'moisture',
    'P': 'pressure',
    'P0': 'pressure',
    'V_TE002': 'soil_temperature',
    'V_TE005': 'soil_temperature',
    'V_TE010': 'soil_temperature',
    'V_TE020': 'soil_temperature',
    'V_TE050': 'soil_temperature',
    'V_TE100': 'soil_temperature',
}

# Prepare dataframes and store in a dictionary
dfs = {data_type: prep_from_anyfolder("basis_23Feb",data_type) for data_type in data_types}

# Concatenate all dataframes
df = pd.concat(dfs.values(), axis=0)
df.columns = df.columns.str.replace(" ","")

columns_to_drop = ['V_N_I', 'VP_STD', 'P_STD', 'TT_STD', 'RF_STD', 'TD_STD', 'RS_IND']
    
    # Drop columns if they exist
for column in columns_to_drop:
    if column in df.columns:
        df = df.drop(columns=[column])
 # Remove quality indicator
df = df.loc[:, ~df.columns.str.startswith('QN_')]
# the esssence of data integration: all measurements taken by the same station at the same time are considered synchronous 
df = df.groupby(["MESS_DATUM", 'STATIONS_ID']).first().reset_index()  

In [38]:
from datetime import datetime as dt

# Assuming df is your DataFrame
min_date = df['MESS_DATUM'].min()
max_date = df['MESS_DATUM'].max()

print(f"Min date: {min_date}")
print(f"Max date: {max_date}")
print(f"Max date to now : {dt.now() - max_date}")

Min date: 2024-01-01 01:00:00
Max date: 2024-02-21 23:00:00
Max date to now : 4 days 14:54:54.607514


In [11]:
# Convert the 'MESS_DATUM' column to datetime
df['MESS_DATUM'] = pd.to_datetime(df['MESS_DATUM'])

# Filter the DataFrame
filtered_df = df[df['MESS_DATUM'] > '2024-02-16']

# Check the length
length = len(filtered_df)

print(length)

715


In [10]:
filtered_df.shape

(715, 21)

In [49]:
from datetime import datetime, timedelta

# Get the current time
now = datetime.now()

# Calculate the time 27 hours ago
time_51_hours_ago = now - timedelta(hours=51)

# Filter the DataFrame
filtered_df = df[df['MESS_DATUM'] >= time_51_hours_ago]
df = filtered_df

In [16]:
# quality check if stations have been sampled
print("Before:", len(df))

# Filter rows where 'STATIONS_ID' is in 'Stations_id' of metadata
df = df[df['STATIONS_ID'].isin(metadata['Stations_id'])]

print("After:", len(df))

Before: 6235
After: 6235


In [22]:
# Assuming df is your DataFrame
memory_usage = df.memory_usage().sum()

print(f"Memory usage: {memory_usage/1024**2} megabytes")

Memory usage: 0.9990768432617188 bytes


## Timestream

In [34]:
# setup for my big timestream ingestor. 
# i would need to authorize a boto3 session first, then metadata dataframe that i use to retrieve geo-location by station id

os.chdir('/home/ec2-user/SageMaker')
metadata = pd.read_csv('Metadata-DWD.csv', sep=',', header=0)
metadata.index  = metadata['Stations_id']

# Set up AWS credentials and region
aws_access_key_id = ''
aws_secret_access_key = ''
region_name = 'eu-central-1'
DATABASE_NAME = 'myclimab'
TABLE_NAME = 'clima-integra'

session = boto3.Session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=region_name
)

In [32]:
print(df.dtypes)
print(df.shape)

MESS_DATUM     datetime64[ns]
STATIONS_ID             int64
F                     float64
D                     float64
SD_SO                 float64
V_N                   float64
FX_911                float64
TT_TU                 float64
RF_TU                 float64
ABSF_STD              float64
TF_STD                float64
R1                    float64
WRTR                  float64
P                     float64
P0                    float64
V_TE002               float64
V_TE005               float64
V_TE010               float64
V_TE020               float64
V_TE050               float64
V_TE100               float64
dtype: object
(715, 21)


In [27]:
# filtering data from 16th Feb
# Convert your date string to a datetime object
date_filter = pd.to_datetime('2024-02-16')

# Filter the DataFrame
df = df[df['MESS_DATUM'] > '2024-02-16']

In [29]:
df.shape

(715, 21)

In [35]:
# AND now the ingestion process begins
def prepare_common_attributes_auto(name):
    metastrip = metadata[metadata['Stationsname'] == name]
    common_attributes = {
        'Dimensions': [
            {'Name': 'name', 'Value': metastrip['Stationsname'].item()},
            {'Name': 'height', 'Value': str(metastrip['Stationshoehe'].item())},
            {'Name': 'latt', 'Value': str(metastrip['geoBreite'].item())},
            {'Name': 'long', 'Value': str(metastrip['geoLaenge'].item())}
        ],    
    }
    return common_attributes
# df goes into prepare records! 
# In the prepare_records function, assign the measure name to each record
def prepare_records(i, df):
    strip = df.iloc[i,:]
    timestamp = str(int(strip['MESS_DATUM'].timestamp() * 1e9))  # Convert to nanoseconds

    multi_val_list = []
    measure_name = None
    for dwd_name, measure in measure_name_dict.items():
        if pd.notna(strip[dwd_name]):
            record = {
                'Name': dwd_name,
                'Value': str(strip[dwd_name]),
                'Type':  'DOUBLE'
            }
            multi_val_list.append(record)
            measure_name = measure

    record = {
        'Time': timestamp,
        'MeasureValues':multi_val_list,
        'TimeUnit': 'NANOSECONDS',
        'MeasureName': measure_name,
        'MeasureValueType': 'MULTI'
    }
    return [record]

def _print_rejected_records_exceptions(err):
    print("RejectedRecords: ", err)
    for rr in err.response["RejectedRecords"]:
        print("Rejected Index " + str(rr["RecordIndex"]) + ": " + rr["Reason"])
        if "ExistingVersion" in rr:
            print("Rejected record existing version: ", rr["ExistingVersion"])

def write_records(write_client, records, common_attributes):
    try:
        result = write_client.write_records(DatabaseName=DATABASE_NAME,
                                            TableName=TABLE_NAME,
                                            CommonAttributes=common_attributes,
                                            Records=records)
        status = result['ResponseMetadata']['HTTPStatusCode']
        print("Processed %d records. WriteRecords HTTPStatusCode: %s" %
              (len(records), status))
    except write_client.exceptions.RejectedRecordsException as err:
        _print_rejected_records_exceptions(err)
    except Exception as err:
        print("Error:", err)


def main():
    print("writing data to database {} table {}".format(
        DATABASE_NAME, TABLE_NAME))
    session = boto3.Session()
    write_client = session.client('timestream-write', config=Config(
        read_timeout=20, max_pool_connections=5000, retries={'max_attempts': 10}))

    log = {}
    for _, row in metadata.iterrows():
        station = row['Stationsname']
        station_id = row['Stations_id']
        common_attributes = prepare_common_attributes_auto(station)
        # create slice dataframe from one station
        station_data_frame = df[df['STATIONS_ID']==station_id]
        records = []
        n_rec = station_data_frame.shape[0]
        log[station] = n_rec
        print("processing",n_rec,"records from weather station ",station)
        
        # pass station df to create records 
        for i in range(n_rec) : 
            record = prepare_records(i, station_data_frame) # prepare from slice df
            records.append(record[0])

        # load data with batches to minimize costs 
        for i in range(0, len(records), 100):
            batch = records[i:i+100]
            try:
                result = write_records(write_client, batch, common_attributes)
            except ClientError as e:
                log[f'Error_{station}_{i}'] = e 
                print('We have found an Error:', e)
    return log 

if __name__ == '__main__':
    mylog = main()

writing data to database myclimab table clima-integra
processing 143 records from weather station  Berlin-Buch
Processed 100 records. WriteRecords HTTPStatusCode: 200
Processed 43 records. WriteRecords HTTPStatusCode: 200
processing 143 records from weather station  Berlin-Dahlem-FU
Processed 100 records. WriteRecords HTTPStatusCode: 200
Processed 43 records. WriteRecords HTTPStatusCode: 200
processing 143 records from weather station  Airport-Berlin-Brandenburg
Processed 100 records. WriteRecords HTTPStatusCode: 200
Processed 43 records. WriteRecords HTTPStatusCode: 200
processing 143 records from weather station  Berlin-Tempelhof
Processed 100 records. WriteRecords HTTPStatusCode: 200
Processed 43 records. WriteRecords HTTPStatusCode: 200
processing 143 records from weather station  Potsdam
Processed 100 records. WriteRecords HTTPStatusCode: 200
Processed 43 records. WriteRecords HTTPStatusCode: 200


In [36]:
mylog

{'Berlin-Buch': 143,
 'Berlin-Dahlem-FU': 143,
 'Airport-Berlin-Brandenburg': 143,
 'Berlin-Tempelhof': 143,
 'Potsdam': 143}

In [19]:
#cursor.execute("DROP DATABASE clima_testerday")
cursor.execute("CREATE DATABASE clima_integra")
cursor.execute("USE clima_integra")  

In [52]:
# Reset the index of the DataFrame
metadata = metadata.reset_index(drop=True)

### preparation of  data for relational purposes

In [60]:
rel_df = df.merge(metadata[['Stationshoehe','geoBreite', 'geoLaenge','Stationsname', 'Stations_id']], 
              how='left', 
              left_on='STATIONS_ID', 
              right_on='Stations_id')
rel_df.drop('Stations_id', axis = 1, inplace = True)
rel_df.columns

Index(['MESS_DATUM', 'STATIONS_ID', 'F', 'D', 'SD_SO', 'V_N', 'FX_911',
       'TT_TU', 'RF_TU', 'ABSF_STD', 'TF_STD', 'R1', 'WRTR', 'P', 'P0',
       'V_TE002', 'V_TE005', 'V_TE010', 'V_TE020', 'V_TE050', 'V_TE100',
       'Stationshoehe', 'geoBreite', 'geoLaenge', 'Stationsname'],
      dtype='object')

In [61]:
rel_df = rel_df[['STATIONS_ID','MESS_DATUM','Stationsname','geoBreite', 'geoLaenge', 'Stationshoehe',   
                 'F', 'D', 'TT_TU', 'RF_TU', 'R1','WRTR','SD_SO', 
                 'V_N', 'FX_911', 'ABSF_STD', 'TF_STD', 'P', 'P0',
                'V_TE002', 'V_TE005', 'V_TE010', 'V_TE020', 'V_TE050', 'V_TE100']]
rel_df = rel_df.replace({np.nan:None})



In [62]:
rel_df.shape

(6235, 25)

In [63]:
# Convert the 'MESS_DATUM' column to datetime
rel_df['MESS_DATUM'] = pd.to_datetime(rel_df['MESS_DATUM'])

# Create a time range for the entire day of '2024-02-16'
start_time = pd.to_datetime('2024-02-16 00:00:00')
end_time = pd.to_datetime('2024-02-16 01:00:00')

# Filter the DataFrame
rel_df = rel_df[(rel_df['MESS_DATUM'] >= start_time) & (rel_df['MESS_DATUM'] < end_time)]

# Print the shape of the filtered DataFrame
print(rel_df.shape)

(5, 25)


In [64]:
rel_df

,STATIONS_ID,MESS_DATUM,Stationsname,geoBreite,geoLaenge,Stationshoehe,F,D,TT_TU,RF_TU,...,ABSF_STD,TF_STD,P,P0,V_TE002,V_TE005,V_TE010,V_TE020,V_TE050,V_TE100
5515,400,2024-02-16,Berlin-Buch,52.6310,13.5021,60,None,None,9.4,95.0,...,8.6,9.0,None,None,None,None,None,None,None,None
5516,403,2024-02-16,Berlin-Dahlem-FU,52.4537,13.3017,51,None,None,9.1,100.0,...,8.9,9.1,1016.3,1006.6,None,7.2,7.5,8.1,6.8,6.2
5517,427,2024-02-16,Airport-Berlin-Brandenburg,52.3807,13.5306,46,4.1,160.0,9.3,94.0,...,8.4,8.8,1016.6,1010.5,None,7.5,8.0,8.1,6.7,6.1
5518,433,2024-02-16,Berlin-Tempelhof,52.4676,13.4020,48,2.3,160.0,10.0,92.0,...,8.6,9.3,1016.5,1010.6,None,8.2,8.6,8.5,6.8,5.8
5519,3987,2024-02-16,Potsdam,52.3812,13.0622,81,5.2,170.0,9.6,92.0,...,8.4,9.0,1016.2,1004.5,None,7.4,8.1,8.2,6.5,5.6


In [65]:
# CREATE TABLE query
table_query = """
    CREATE TABLE DWDtest1 (
        id INT AUTO_INCREMENT,
        station_id INT,
        datetime DATETIME,
        name VARCHAR(255),
        lattitude FLOAT,
        longitude FLOAT,
        height FLOAT,
        wind_speed FLOAT,
        wind_direct FLOAT,
        temp FLOAT,
        humidity FLOAT,
        prec_mm FLOAT,
        prec_form INT,
        sun_dur FLOAT,
        cloudy TINYINT,
        extreme_wind_speed INT,
        humidity_abs FLOAT,
        temp_wet_bulb FLOAT,
        pres_sealvl FLOAT,
        pres FLOAT,
        soil_temp_2 FLOAT,
        soil_temp_5 FLOAT,
        soil_temp_10 FLOAT,
        soil_temp_20 FLOAT,
        soil_temp_50 FLOAT,
        soil_temp_100 FLOAT,
        PRIMARY KEY (id)
    );
"""

# INSERT INTO query
insert_query = '''
    INSERT INTO DWDtest1 (
        station_id, 
        datetime, 
        name, 
        lattitude, 
        longitude, 
        height, 
        wind_speed, 
        wind_direct, 
        temp, 
        humidity, 
        prec_mm, 
        prec_form, 
        sun_dur, 
        cloudy, 
        extreme_wind_speed, 
        humidity_abs, 
        temp_wet_bulb, 
        pres_sealvl, 
        pres, 
        soil_temp_2, 
        soil_temp_5, 
        soil_temp_10, 
        soil_temp_20, 
        soil_temp_50, 
        soil_temp_100
    ) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
'''


#cursor.execute(table_query)
# Insert data batch-wise
batch_size = 1000
batches = [rel_df[i:i + batch_size] for i in range(0, rel_df.shape[0], batch_size)]
# Corrected data_tuples creation
for batch in batches:
    data_tuples = [tuple(row[col] for col in rel_df.columns) for index, row in batch.iterrows()]

    # Debugging
    print(f"Number of placeholders in SQL query: {insert_query.count('%s')}")
    print(f"Number of elements in data_tuple: {len(data_tuples[0])}")
    cursor.executemany(insert_query, data_tuples)

    conn.commit()

Number of placeholders in SQL query: 25
Number of elements in data_tuple: 25


In [22]:
cursor.fetchall()

[]

In [33]:
# Replace 'your_table_name' and 'your_date_column' with your actual table name and date column name
cursor.execute("DELETE FROM DWD WHERE datetime >= '2024-02-16'")

Reporting tool to analyze the distribution among weather stations and dates range

In [ ]:
observation_counts = {}

for data_type, data_frame in dfs.items():
    observation_counts[data_type] = data_frame.groupby('STATIONS_ID').size()

# Print the results
for data_type, counts in observation_counts.items():
    print(f"\nMeasurement type: {data_type}")
    print(counts)

In [15]:
# Report shape and date range for each dataframe
for name, dataframe in dfs.items():
    print(f"DataFrame: {name}")
    print(f"Shape: {dataframe.shape}")
    print(f"Earliest date: {dataframe['MESS_DATUM'].min()}")
    print(f"Latest date: {dataframe['MESS_DATUM'].max()}")
    print("\n")

DataFrame: wind
Shape: (3597, 5)
Earliest date: 2024-01-01 01:00:00
Latest date: 2024-02-19 23:00:00


DataFrame: sun
Shape: (3597, 4)
Earliest date: 2024-01-01 01:00:00
Latest date: 2024-02-19 23:00:00


DataFrame: cloudiness
Shape: (3597, 5)
Earliest date: 2024-01-01 01:00:00
Latest date: 2024-02-19 23:00:00


DataFrame: extreme_wind
Shape: (4796, 4)
Earliest date: 2024-01-01 01:00:00
Latest date: 2024-02-19 23:00:00


DataFrame: air_temperature
Shape: (5995, 5)
Earliest date: 2024-01-01 01:00:00
Latest date: 2024-02-19 23:00:00


DataFrame: moisture
Shape: (5990, 10)
Earliest date: 2024-01-01 01:00:00
Latest date: 2024-02-19 23:00:00


DataFrame: precipitation
Shape: (5995, 6)
Earliest date: 2024-01-01 01:00:00
Latest date: 2024-02-19 23:00:00


DataFrame: pressure
Shape: (4796, 5)
Earliest date: 2024-01-01 01:00:00
Latest date: 2024-02-19 23:00:00


DataFrame: soil_temperature
Shape: (4796, 9)
Earliest date: 2024-01-01 01:00:00
Latest date: 2024-02-19 23:00:00


